<a href="https://colab.research.google.com/github/maidacundo/test_referees_optimization/blob/main/test_2_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [406]:
!pip install ortools

In [407]:
def make_day(num_teams, day):
    # using circle algorithm, https://en.wikipedia.org/wiki/Round-robin_tournament#Scheduling_algorithm
    assert not num_teams % 2, "Number of teams must be even!"
    # generate list of teams
    lst = list(range(num_teams))
    # rotate
    day %= (num_teams - 1)  # clip to 0 .. num_teams - 2
    if day:                 # if day == 0, no rotation is needed (and using -0 as list index will cause problems)
        lst = lst[:1] + lst[-day:] + lst[1:-day]
    # pair off - zip the first half against the second half reversed
    half = num_teams // 2
    return list(zip(lst[:half], lst[half:][::-1]))

def make_schedule(num_teams):
    """
    Produce a double round-robin schedule
    """
    # number of teams must be even
    if num_teams % 2:
        num_teams += 1  # add a dummy team for padding

    # build first round-robin
    schedule = [make_day(num_teams, day) for day in range(num_teams - 1)]
    # generate second round-robin by swapping home,away teams
    swapped = [[(away, home) for home, away in day] for day in schedule]

    return schedule + swapped

In [408]:
# modellazione use case serie a

from ortools.sat.python import cp_model

model = cp_model.CpModel()

# informazioni riguardo al modello di campionato

num_teams = 4
num_referee = 40
calendar = make_schedule(num_teams)
calendar = calendar[:5]
num_days = len(calendar)

min_num_referee = 3

squadre = []
for i in range(num_teams):
  squadre.append(i)

In [409]:
# lista per tenere traccia di quante volte un arbitro ha arbitrato una partita in passato

referee_history = []
for i in range(num_referee):
  referee_history.append([])
  for s in range(len(squadre)):
    referee_history[i].append(1)

referee_history[0][0] = 0
referee_history[1][1] = 0
referee_history[2][2] = 0
referee_history[3][3] = 0
referee_history[0][1] = 0
referee_history[1][2] = 0
referee_history[2][3] = 0

referee_history

[[0, 0, 1, 1],
 [1, 0, 0, 1],
 [1, 1, 0, 0],
 [1, 1, 1, 0],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1]]

In [410]:
# matrice che contiene le predizioni sugli arbitri

x = []
for i in range(num_referee):
    r = []
    for day in range(num_days):
        num_matches = len(calendar[day])
        d = []
        for j in range(num_matches):
            d.append(model.NewIntVar(0, 1, 'referee{}_day{}_match{}'.format(i, day, j)))
        r.append(d)
    x.append(r)

In [411]:
# definisco le constraints del modello 

# Ogni arbitro deve arbitrare almeno una volta
for i in range(num_referee):
    pass
    # model.AddAtLeastOne(x[i][d][j] for d in range(num_days) for j in range(num_matches))

# Ogni arbitro è assegnato al max ad una partita per giorno
for i in range(num_referee):
    for d in range(num_days):
        model.AddAtMostOne(x[i][d][j] for j in range(num_matches))

# Ogni partita ha 3 arbitri
for d in range(num_days):
    for j in range(num_matches):
        model.Add(sum([x[i][d][j] for i in range(num_referee)]) == min_num_referee)


In [412]:
# calcolo dei match totali arbitrati da ogni singolo arbitro

total_matches_per_referee = []

for i in range(num_referee):
    total_matches = 0
    for d in range(num_days):
        total_matches += sum(x[i][d])
    total_matches_per_referee.append(total_matches)
    matches_differences = []
    
all_variables = []

for i in range(len(total_matches_per_referee)):
    c = []
    for j in range(len(total_matches_per_referee)):
        if i != j:
            diff_var = model.NewIntVar(0, num_days, f'matches_diff_{i}_{j}')
            abs_diff = model.AddAbsEquality(diff_var, (total_matches_per_referee[i] - total_matches_per_referee[j]))
            c.append(diff_var)
            all_variables.append(diff_var)
    matches_differences.append(c)

# aggiungo una decision stategy (euristica) per migliorare le performance
model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_LOWER_HALF)

# matches_differences[referee] ritorna la lista di differenze tra quell'arbitro e gli altri
total_differences_per_referee = []
for i in range(len(matches_differences)):
    total_differences_per_referee.append(sum(matches_differences[i]))
  
total_diff_per_referre_to_minimize = sum(total_differences_per_referee)

In [413]:
# calcolo delle squadre arbitrate da ogni singolo arbitro

referee_per_team_matches = []

for i in range(num_referee):
  teams_per_referee = []
  for s in squadre:
    matches_per_team = []
    for d in range(num_days):
      for m in calendar[d]:
        if s in m:
          j = calendar[d].index(m)
          matches_per_team.append(x[i][d][j])
    
    try:
      previous_matches_number = referee_history[i][s]
    except IndexError:
      previous_matches_number = 0

    # commentare per utilizzare lo storico delle partite
    previous_matches_number = 0
    
    teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
  referee_per_team_matches.append(teams_per_referee)

team_per_referee_matches = [list(i) for i in zip(*referee_per_team_matches)]

# referee_per_team_matches[referee][team] ritorna la somma di partite arbitrate da quell'arbitro per quella squadra
# team_per_referee_matches[team][referee] ritorna la somma di partite arbitrate da quell'arbitro per quella squadra

In [414]:
team_per_referee_matches[0]

[Sum(Sum(Sum(Sum(referee0_day0_match0(0..1), referee0_day1_match0(0..1)), referee0_day2_match0(0..1)), referee0_day3_match0(0..1)), referee0_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee1_day0_match0(0..1), referee1_day1_match0(0..1)), referee1_day2_match0(0..1)), referee1_day3_match0(0..1)), referee1_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee2_day0_match0(0..1), referee2_day1_match0(0..1)), referee2_day2_match0(0..1)), referee2_day3_match0(0..1)), referee2_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee3_day0_match0(0..1), referee3_day1_match0(0..1)), referee3_day2_match0(0..1)), referee3_day3_match0(0..1)), referee3_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee4_day0_match0(0..1), referee4_day1_match0(0..1)), referee4_day2_match0(0..1)), referee4_day3_match0(0..1)), referee4_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee5_day0_match0(0..1), referee5_day1_match0(0..1)), referee5_day2_match0(0..1)), referee5_day3_match0(0..1)), referee5_day4_match0(0..1)),
 Sum(Sum(Sum(Sum(referee6_day0_mat

In [415]:
all_variables = []
t = []

for i in range(len(referee_per_team_matches)):
  c = []
  for s in squadre:
    other_teams_sum = sum(team_per_referee_matches[s][:i] + team_per_referee_matches[s][i+1 :])
    diff_var = model.NewIntVar(0, 1000, f'matches_diff_{i}_team_{s}') # todo controllare meglio up e lb!
    abs_diff = model.AddAbsEquality(diff_var, (referee_per_team_matches[i][s] - other_teams_sum))
    c.append(diff_var)
    all_variables.append(diff_var)
  t.append(c)

team_diff_sum = []
for i in range(num_referee):
  team_diff_sum.append(sum([t[i][s] for s in squadre]))

team_diff_per_referre_to_minimize = sum(team_diff_sum)
team_diff_per_referre_to_minimize

Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(matches_diff_0_team_0(0..1000), matches_diff_0_team_1(0..1000)), matches_diff_0_team_2(0..1000)), matches_diff_0_team_3(0..1000)), Sum(Sum(Sum(matches_diff_1_team_0(0..1000), matches_diff_1_team_1(0..1000)), matches_diff_1_team_2(0..1000)), matches_diff_1_team_3(0..1000))), Sum(Sum(Sum(matches_diff_2_team_0(0..1000), matches_diff_2_team_1(0..1000)), matches_diff_2_team_2(0..1000)), matches_diff_2_team_3(0..1000))), Sum(Sum(Sum(matches_diff_3_team_0(0..1000), matches_diff_3_team_1(0..1000)), matches_diff_3_team_2(0..1000)), matches_diff_3_team_3(0..1000))), Sum(Sum(Sum(matches_diff_4_team_0(0..1000), matches_diff_4_team_1(0..1000)), matches_diff_4_team_2(0..1000)), matches_diff_4_team_3(0..1000))), Sum(Sum(Sum(matches_diff_5_team_0(0..1000), matches_diff_5_team_1(0..1000)), matches_diff_5_team_2(0..1000)), matches_diff_5_

In [416]:
# lista contenente tutti i valori da minimizzare

values_to_minimize = []
# values_to_minimize.append(team_diff_per_referre_to_minimize * 0.7)
values_to_minimize.append(total_diff_per_referre_to_minimize)

In [417]:
# model.Minimize(total_diff_per_referre_to_minimize)
model.Maximize(team_diff_per_referre_to_minimize)

In [418]:
print(model.ModelStats())

optimization model '':
Search strategy: on 1560 variables, CHOOSE_LOWEST_MIN, SELECT_LOWER_HALF
#Variables: 2120 (#ints:160 in objective)
  - 400 in [0,1]
  - 1560 in [0,5]
  - 160 in [0,1000]
#kAtMostOne: 200 (#literals: 400)
#kLinMax: 1720
#kLinearN: 10 (#terms: 400)


In [419]:
calendar

[[(0, 3), (1, 2)],
 [(0, 2), (3, 1)],
 [(0, 1), (2, 3)],
 [(3, 0), (2, 1)],
 [(2, 0), (1, 3)]]

In [420]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, print_boolean = False):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.print_boolean = print_boolean

    def on_solution_callback(self):
        self.__solution_count += 1
        
        print(self.__solution_count)
        print(f'Differenza totale per partite arbitrate: {self.Value(total_diff_per_referre_to_minimize)}')
        print(f'Differenza totale per squadre arbitrate: {self.Value(team_diff_per_referre_to_minimize)}')
        print('-'* 30)

        if self.print_boolean:
          print('POSSIBILE SOLUZIONE')
          match_per_referee = [0] * num_referee

          for d in range(num_days):
              print('-' * 30)
              print(f'Day {d}:\n')
              num_matches = len(calendar[d])
              for i in range(num_referee):
                  for j in range(num_matches):
                      if self.BooleanValue(self.__variables[i][d][j]):
                          match_per_referee[i] += 1
                          print(
                              f'Referee {i} assigned to match {j}')
          print()
          print('TOTALI PARTITE PER ARBITRO')
          for i in range(num_referee):
            print(f'Referee {i}: {match_per_referee[i]}')
          print()

    def solution_count(self):
        return self.__solution_count


In [421]:
import multiprocessing

cpu_count = multiprocessing.cpu_count()
print(f'runnin on {cpu_count} CPUs')

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(x)

#solver.parameters.enumerate_all_solutions = True
solver.parameters.num_search_workers = cpu_count

solver.parameters.max_time_in_seconds = 600.0

status = solver.Solve(model, solution_printer)

runnin on 2 CPUs
1
Differenza totale per partite arbitrate: 600
Differenza totale per squadre arbitrate: 2280
------------------------------


In [422]:
print(f'conflicts: {solver.NumConflicts()}')
print(f'braches: {solver.NumBranches()}')

conflicts: 44
braches: 1489


In [423]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('POSSIBILE SOLUZIONE')

    match_per_referee = [0] * num_referee

    for i in range(num_referee):
        print('-' * 30)
        print(f'Referee {i}:\n')
        num_matches = len(calendar[d])
        for d in range(num_days):
            for j in range(num_matches):
                if solver.BooleanValue(x[i][d][j]):
                    match_per_referee[i] += 1
                    print(
                        f'Referee {i} assigned to match {calendar[d][j]}')
    print()
    print('TOTALI PARTITE PER ARBITRO')
    for i in range(num_referee):
      print(f'Referee {i}: {match_per_referee[i]}')
    
    print(f'Differenza totale per partite arbitrate: {solver.Value(total_diff_per_referre_to_minimize)}')
    print(f'Differenza totale per squadre arbitrate: {solver.Value(team_diff_per_referre_to_minimize)}')

    print()
    print('TOTALI PARTITE PER SQUADRA')
    for i in range(num_referee):
      print(f'Referee {i}: {solver.Value(team_diff_sum[i])}')
    print()
else:
    print('No solution found.')

POSSIBILE SOLUZIONE
------------------------------
Referee 0:

Referee 0 assigned to match (2, 3)
------------------------------
Referee 1:

Referee 1 assigned to match (2, 0)
------------------------------
Referee 2:

Referee 2 assigned to match (3, 0)
------------------------------
Referee 3:

Referee 3 assigned to match (1, 3)
------------------------------
Referee 4:

Referee 4 assigned to match (0, 2)
------------------------------
Referee 5:

Referee 5 assigned to match (1, 2)
------------------------------
Referee 6:

Referee 6 assigned to match (3, 1)
------------------------------
Referee 7:

Referee 7 assigned to match (3, 0)
------------------------------
Referee 8:

Referee 8 assigned to match (0, 3)
------------------------------
Referee 9:

Referee 9 assigned to match (2, 3)
------------------------------
Referee 10:

Referee 10 assigned to match (2, 3)
------------------------------
Referee 11:

------------------------------
Referee 12:

Referee 12 assigned to match (1,

In [424]:
# calcolo delle squadre arbitrate da ogni singolo arbitro

referee_matches_per_team = []

for i in range(num_referee):
  teams_per_referee = []
  for s in squadre:
    matches_per_team = []
    for d in range(num_days):
      for m in calendar[d]:
        if s in m:
          j = calendar[d].index(m)
          matches_per_team.append(x[i][d][j])
    
    try:
      previous_matches_number = referee_history[i][s]
    except IndexError:
      previous_matches_number = 0

    # commentare per utilizzare lo storico delle partite
    previous_matches_number = 0
    
    teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
  referee_matches_per_team.append(teams_per_referee)

# referee_matches_per_team[referee][team] ritorna la somma di partite arbitrate da quell'arbitro per quella squadra

team_diff = []
all_variables = []

for i in range(len(referee_matches_per_team)):
  t = []
  for s in squadre:
    c = []
    for j in range(len(referee_matches_per_team)):
      if i != j:
        diff_var = model.NewIntVar(0, num_days, f'matches_diff_{i}_{j}_team_{s}') # todo controllare meglio up e lb!
        abs_diff = model.AddAbsEquality(diff_var, (referee_matches_per_team[i][s] - referee_matches_per_team[j][s]))
        c.append(diff_var)
        all_variables.append(diff_var)
    
    t.append(sum(c))
  team_diff.append(sum(t))

# aggiungo una decision stategy (euristica) per migliorare le performance
model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)

# v[referee] ritorna la differenza totate di quell'arbitro per tutti i team rispetto a tutti gli altri arbitri

team_diff_per_referre_to_minimize = sum(team_diff)
team_diff_per_referre_to_minimize

Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(matches_diff_0_1_team_0(0..5), matches_diff_0_2_team_0(0..5)), matches_diff_0_3_team_0(0..5)), matches_diff_0_4_team_0(0..5)), matches_diff_0_5_team_0(0..5)), matches_diff_0_6_team_0(0..5)), matches_diff_0_7_team_0(0..5)), matches_diff_0_8_team_0(0..5)), matches_diff_0_9_team_0(0..5)), matches_diff_0_10_team_0(0..5)), matches_diff_0_11_team_0(0..5)), matches_diff_0_12_team_0(0..5)), matches_diff_0_13_team_0(0..5)), matches_diff_0_14_team_0(0..5)), matches_diff_0_15_team_0(0..5)), matches_diff_0_16_team_0(0..5)), matches_diff_0_17_team_0(0..5)), matches_diff_0_18_team_0(0..5)), matches_diff_0_19_team_0(0..5)), matches_diff_0_20_team_0(0..5)), matches_diff_0_21_team_0(0..5)

In [425]:
print('TOTALI PARTITE PER SQUADRA')
for i in range(num_referee):
  print(f'Referee {i}: {solver.Value(team_diff[i])} - {solver.Value(team_diff_sum[i])}')

TOTALI PARTITE PER SQUADRA


IndexError: ignored

In [ ]:
team_diff[0]